In [ ]:
import requests
import pandas as pd
import sqlite3


url = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=gc_obo_gare_origine_r_name+%3D+%22Paris%22+AND+(date%3D'2019'+OR+date%3D'2020'+OR+date+%3D'2021'+OR+date%3D'2022')&sort=date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c&timezone=Europe%2FParis&rows=-1"


response = requests.get(url)
data = response.json()
facet = data["parameters"]["facet"]


#---------------------------------------------------------------------------------------------------------------
# Extraire les valeurs de facet
facets = facet
date = facets[0]
gc_obo_date_heure_restitution_c = facets[1]
gc_obo_gare_origine_r_name = facets[2]
gc_obo_nature_c = facets[3]
gc_obo_type_c = facets[4]
gc_obo_nom_recordtype_sc_c = facets[5]

# Extraire les champs des enregistrements
records = data["records"]
record_fields = []
for record in records:
    record_fields.append(record["fields"])

#print(record_fields)

# Créer un DataFrame à partir des champs des enregistrements
df = pd.DataFrame(record_fields)


# Écrire le DataFrame dans un fichier CSV
df.to_csv("objets-trouves.csv", index=False)



connexion = sqlite3.connect('bdd.db')

df.to_sql('objets_trouves', connexion,if_exists='replace', index=False)

connexion.close()


In [49]:
import requests

url_gares = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=referentiel-gares-voyageurs&q=&sort=gare_alias_libelle_noncontraint&rows=-1"


response = requests.get(url_gares)

data_list = []

if response.status_code == 200:
    data = response.json()
    for record in data['records']:
        gare_alias_libelle_noncontraint = record['fields']['gare_alias_libelle_noncontraint']
        if 'wgs_84' in record['fields']:
            latitude = record['fields']['wgs_84'][0]
            longitude = record['fields']['wgs_84'][1]
            data_list.append([gare_alias_libelle_noncontraint, latitude, longitude])

            
else:
    print("Une erreur s'est produite lors de la requête à l'API.")
    
df_data = pd.DataFrame(data_list, columns=[
    "Gare",
    "Latitude",
    "Longitude"
])

df_data.to_csv('df_data.csv')

import sqlite3
import pandas as pd 

connexion = sqlite3.connect('bdd.db')

df_data.to_sql('coordonnees_gares', connexion,if_exists='replace', index=False)

connexion.close()
